In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Set the path for the train and test data directories
train_dir = '/path/to/fer2013/train'
test_dir = '/path/to/fer2013/test'

# Set the batch size and number of epochs
batch_size = 32
epochs = 50

# Define the input shape of the images
input_shape = (48, 48, 1)

# Define the model architecture
input_layer = Input(shape=input_shape)
x = Conv2D(32, kernel_size=(3, 3), activation='relu')(input_layer)
x = MaxPooling2D(pool_size=(2, 2))(x)
x = Conv2D(64, kernel_size=(3, 3), activation='relu')(x)
x = MaxPooling2D(pool_size=(2, 2))(x)
x = Conv2D(128, kernel_size=(3, 3), activation='relu')(x)
x = MaxPooling2D(pool_size=(2, 2))(x)
x = Flatten()(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.5)(x)
output_layer = Dense(7, activation='softmax')(x)

model = Model(inputs=input_layer, outputs=output_layer)

# Compile the model with categorical crossentropy loss and Adam optimizer
model.compile(loss='categorical_crossentropy',
              optimizer=Adam(lr=0.001),
              metrics=['accuracy'])

# Define data generators for training and testing
train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(48, 48),
    color_mode='grayscale',
    batch_size=batch_size,
    class_mode='categorical')

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(48, 48),
    color_mode='grayscale',
    batch_size=batch_size,
    class_mode='categorical')

# Set up early stopping callback to prevent overfitting
early_stopping = EarlyStopping(monitor='val_loss', patience=5)

# Train the model on the train data and evaluate it on the test data
model.fit(train_generator,
          epochs=epochs,
          validation_data=test_generator,
          callbacks=[early_stopping])


In [ ]:
import pandas as pd
import numpy as np
import cv2

# Load the dataset
data = pd.read_csv('fer2013.csv')

# Preprocess the data
X_train, y_train, X_test, y_test = [], [], [], []
for i in range(len(data)):
    emotion, pixels, usage = data.iloc[i,:].values
    img = np.array([int(x) for x in pixels.split()]).reshape(48,48)
    if usage == 'Training':
        X_train.append(cv2.resize(img.astype('uint8'), (width, height)))
        y_train.append(emotion)
    elif usage == 'PublicTest':
        X_test.append(cv2.resize(img.astype('uint8'), (width, height)))
        y_test.append(emotion)

# Convert to numpy arrays and normalize pixel values
X_train = np.array(X_train).astype('float32') / 255.0
y_train = np.array(y_train)
X_test = np.array(X_test).astype('float32') / 255.0
y_test = np.array(y_test)

# Convert labels to categorical
from tensorflow.keras.utils import to_categorical
y_train = to_categorical(y_train, num_classes)
y_test = to_categorical(y_test, num_classes)

# Split training data into training and validation sets
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)


In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Dropout, concatenate
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from skimage.feature import hog, local_binary_pattern
from skimage.filters import gabor_kernel

# Define function to extract LBP features
def extract_lbp_features(image):
    # define the LBP parameters
    radius = 3
    n_points = 8 * radius
    method = 'uniform'
    
    # compute the LBP features for the image
    lbp = local_binary_pattern(image, n_points, radius, method)
    hist, _ = np.histogram(lbp.ravel(), bins=np.arange(0, n_points + 3), range=(0, n_points + 2))
    hist = hist.astype("float")
    hist /= (hist.sum() + 1e-7)
    
    # return the features as a 1D NumPy array
    return hist.ravel()

# Define function to extract HOG features
def extract_hog_features(image):
    # define the HOG parameters
    orientations = 9
    pixels_per_cell = (8, 8)
    cells_per_block = (2, 2)
    visualize = False
    
    # compute the HOG features for the image
    hog_features = hog(image, orientations=orientations, pixels_per_cell=pixels_per_cell,
                       cells_per_block=cells_per_block, visualize=visualize)
    
    # return the features as a 1D NumPy array
    return hog_features.ravel()

# Define function to extract Gabor features
def extract_gabor_features(image):
    # define the Gabor parameters
    freqs = [0.05, 0.1, 0.2]
    thetas = [0, np.pi/4, np.pi/2, 3*np.pi/4]
    kernels = []
    
    # create the Gabor kernels
    for freq in freqs:
        for theta in thetas:
            kernel = np.real(gabor_kernel(freq, theta=theta))
            kernels.append(kernel)
            
    # compute the Gabor features for the image
    gabor_features = np.zeros((len(kernels), 2), dtype=np.double)
    for k, kernel in enumerate(kernels):
        filtered = ndi.convolve(image, kernel, mode='wrap')
        gabor_features[k, 0] = filtered.mean()
        gabor_features[k, 1] = filtered.var()
    
    # return the features as a 1D NumPy array
    return gabor_features.ravel()

# Define the model architecture
input_shape = (width, height, 1)
input_layer = Input(shape=input_shape)

# Convolutional layers for image data
x = Conv2D(32, kernel_size=(3,3), activation='relu')(input_layer)
x = MaxPooling2D()(x)
x = Conv2D(64, kernel_size=(3,3), activation='relu')(x)
x = MaxPooling2D()(x)
x = Conv2D(128, kernel_size=(3,3), activation='relu')(x)
x = MaxPooling2D()(x)
x = Flatten()(x)

# Fully connected layers for LBP, HOG, and Gabor features
lbp_input = Input(shape=(26,), name='lbp_input')
hog_input = Input(shape=(3780,), name='hog_input')
gabor_input = Input(shape=(15,), name='gabor_input')


# Concatenate the convolutional and fully connected layers
merged = tf.keras.layers.concatenate([x, lbp_dense, hog_dense, gabor_dense])

# Add additional fully connected layers
merged = Dense(256, activation='relu')(merged)
merged = Dropout(0.5)(merged)
merged = Dense(128, activation='relu')(merged)
merged = Dropout(0.5)(merged)
output_layer = Dense(num_classes, activation='softmax')(merged)

# Define the complete model
model = Model(inputs=[input_layer, lbp_input, hog_input, gabor_input], outputs=output_layer)

# Define the optimizer, loss function, and metrics
optimizer = Adam(learning_rate=0.001)
loss_function = 'categorical_crossentropy'
metrics = ['accuracy']

# Compile the model
model.compile(optimizer=optimizer, loss=loss_function, metrics=metrics)

# Define the early stopping callback
early_stopping = EarlyStopping(patience=5)

# Define the data generators for training and validation
train_datagen = ImageDataGenerator(rescale=1./255, rotation_range=10, width_shift_range=0.1,
height_shift_range=0.1, shear_range=0.1, zoom_range=0.1,
horizontal_flip=True, vertical_flip=False, fill_mode='nearest',
preprocessing_function=extract_lbp_features)

val_datagen = ImageDataGenerator(rescale=1./255, preprocessing_function=extract_lbp_features)

train_generator = train_datagen.flow_from_directory(train_dir, target_size=(width, height),
color_mode='grayscale', batch_size=batch_size,
class_mode='categorical', shuffle=True)

val_generator = val_datagen.flow_from_directory(val_dir, target_size=(width, height),
color_mode='grayscale', batch_size=batch_size,
class_mode='categorical', shuffle=False)

# Train the model
history = model.fit(train_generator, epochs=num_epochs, validation_data=val_generator,
callbacks=[early_stopping], verbose=1)


In [ ]:
## Whole code

In [ ]:
import os
import numpy as np
from skimage import io, color, transform, feature, filters
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Dropout, concatenate
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from skimage.feature import hog, local_binary_pattern
from skimage.filters import gabor_kernel
from scipy import ndimage as ndi

# Define function to extract LBP features
def extract_lbp_features(image):
    # define the LBP parameters
    radius = 3
    n_points = 8 * radius
    method = 'uniform'
    
    # compute the LBP features for the image
    lbp = local_binary_pattern(image, n_points, radius, method)
    hist, _ = np.histogram(lbp.ravel(), bins=np.arange(0, n_points + 3), range=(0, n_points + 2))
    hist = hist.astype("float")
    hist /= (hist.sum() + 1e-7)
    
    # return the features as a 1D NumPy array
    return hist.ravel()

# Define function to extract HOG features
def extract_hog_features(image):
    # define the HOG parameters
    orientations = 9
    pixels_per_cell = (8, 8)
    cells_per_block = (2, 2)
    visualize = False
    
    # compute the HOG features for the image
    hog_features = hog(image, orientations=orientations, pixels_per_cell=pixels_per_cell,
                       cells_per_block=cells_per_block, visualize=visualize)
    
    # return the features as a 1D NumPy array
    return hog_features.ravel()

# Define function to extract Gabor features
def extract_gabor_features(image):
    # define the Gabor parameters
    freqs = [0.05, 0.1, 0.2]
    thetas = [0, np.pi/4, np.pi/2, 3*np.pi/4]
    kernels = []
    
    # create the Gabor kernels
    for freq in freqs:
        for theta in thetas:
            kernel = np.real(gabor_kernel(freq, theta=theta))
            kernels.append(kernel)
            
    # compute the Gabor features for the image
    gabor_features = np.zeros((len(kernels), 2), dtype=np.double)
    for k, kernel in enumerate(kernels):
        filtered = ndi.convolve(image, kernel, mode='wrap')
        gabor_features[k, 0] = filtered.mean()
        gabor_features[k, 1] = filtered.var()
    
    # return the features as a 1D NumPy array
    return gabor_features.ravel()

# Define function to load images and extract features
def load_images(folder):
    image_filenames = os.listdir(folder)
    images = []
    labels = []

    
    for filename in image_filenames:
        label = int(filename.split('_')[0])
        image = io.imread(os.path.join(folder, filename))
        image = transform.resize(image, (48, 48))
        gray_image = color.rgb2gray(image)
        
        # extract features from the image
        lbp_features = extract_lbp_features(gray_image)
        hog_features = extract_hog_features(gray_image)
        gabor_features = extract_gabor_features(gray_image)
        
        # concatenate the features and original image data
        features = np.concatenate((lbp_features, hog_features, gabor_features))
        
        # add to the lists of images and labels
        images.append(image)
        labels.append(label)
    
    return np.array(images), np.array(labels), np.array(features)



In [ ]:
#Load training and testing data
train_images, train_labels = load_images(train_dir)
test_images, test_labels = load_images(test_dir)

#Reshape data for use in the neural network
train_images = train_images.reshape(-1, feature_size)
test_images = test_images.reshape(-1, feature_size)

#Normalize pixel values to be between 0 and 1
train_images = train_images.astype('float32') / 255
test_images = test_images.astype('float32') / 255

#Convert labels to categorical variables
train_labels = to_categorical(train_labels, num_classes=7)
test_labels = to_categorical(test_labels, num_classes=7)

#Define the neural network architecture
input_layer = Input(shape=(feature_size,))
x = Dense(128, activation='relu')(input_layer)
x = Dropout(0.2)(x)
x = Dense(64, activation='relu')(x)
x = Dropout(0.2)(x)
x = Dense(32, activation='relu')(x)
output_layer = Dense(7, activation='softmax')(x)

model = Model(inputs=input_layer, outputs=output_layer)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

#Train the model
model.fit(train_images, train_labels, epochs=20, batch_size=64, validation_data=(test_images, test_labels))

In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Dropout, Flatten, concatenate

# Define inputs for image and feature data
input_image = Input(shape=(48, 48, 1))
input_features = Input(shape=(lbp_features.shape[0] + hog_features.shape[0] + gabor_features.shape[0],))

# Convolutional layers for image data
x = Conv2D(32, kernel_size=(3,3), activation='relu')(input_image)
x = MaxPooling2D()(x)
x = Conv2D(64, kernel_size=(3,3), activation='relu')(x)
x = MaxPooling2D()(x)
x = Conv2D(128, kernel_size=(3,3), activation='relu')(x)
x = MaxPooling2D()(x)
x = Flatten()(x)

# Fully connected layers for feature data
y = Dense(256, activation='relu')(input_features)
y = Dropout(0.5)(y)
y = Dense(128, activation='relu')(y)
y = Dropout(0.5)(y)
y = Dense(64, activation='relu')(y)

# Concatenate image and feature data and add more fully connected layers
combined = concatenate([x, y])
z = Dense(64, activation='relu')(combined)
z = Dropout(0.5)(z)
z = Dense(32, activation='relu')(z)
output = Dense(num_classes, activation='softmax')(z)

# Define model with inputs and outputs
model = Model(inputs=[input_image, input_features], outputs=output)

# Compile model with loss function and optimizer
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


In [1]:
import os
import numpy as np
from skimage import io, color, transform
from skimage.feature import local_binary_pattern, hog, greycomatrix, greycoprops
from keras.models import Model
from keras.layers import Input, Conv2D, MaxPooling2D, Dense, Dropout, Flatten, concatenate

# Define functions to extract LBP, HOG, and Gabor features
def extract_lbp_features(image):
    lbp_radius = 3
    lbp_n_points = 8 * lbp_radius
    lbp_method = 'uniform'
    lbp_histogram_bins = 26
    
    lbp = local_binary_pattern(image, lbp_n_points, lbp_radius, method=lbp_method)
    lbp_histogram, _ = np.histogram(lbp, bins=lbp_histogram_bins, range=(0, lbp_histogram_bins), density=True)
    
    return lbp_histogram

def extract_hog_features(image):
    hog_orientations = 8
    hog_pixels_per_cell = (6, 6)
    hog_cells_per_block = (2, 2)
    
    hog_features = hog(image, orientations=hog_orientations, pixels_per_cell=hog_pixels_per_cell, cells_per_block=hog_cells_per_block)
    
    return hog_features

def extract_gabor_features(image):
    gabor_frequencies = [0.05, 0.25]
    gabor_angles = [0, np.pi/4, np.pi/2, 3*np.pi/4]
    
    gabor_features = []
    for frequency in gabor_frequencies:
        for angle in gabor_angles:
            real, imag = greycomatrix(image, [1], [angle], levels=256, symmetric=True, normed=True)
            gabor_response = np.abs(np.fft.fftshift(np.fft.fft2(real * np.cos(angle) + imag * np.sin(angle))))
            gabor_filtered = gabor_response * np.exp(-0.5 * ((gabor_response/gabor_response.mean())**2))
            gabor_features.append(gabor_filtered.mean())
    
    return gabor_features

# Define function to load images and extract features
def load_images(folder):
    image_filenames = os.listdir(folder)
    images = []
    labels = []
    for filename in image_filenames:
        label = int(filename.split('_')[0])
        image = io.imread(os.path.join(folder, filename))
        image = transform.resize(image, (48, 48))
        gray_image = color.rgb2gray(image)
        
        # extract features from the image
        lbp_features = extract_lbp_features(gray_image)
        hog_features = extract_hog_features(gray_image)
        gabor_features = extract_gabor_features(gray_image)
        
        # concatenate the features and original image data
        features = np.concatenate((lbp_features, hog_features, gabor_features))
        images.append(image)
        labels.append(label)
        
    return np.array(images), np.array(features), np.array(labels)


ModuleNotFoundError: No module named 'skimage'

In [ ]:
# Load the training and testing data
train_folder = 'C:/Users/HP/Desktop/face_emotion_github/Data/Fer2013/archive/train/'
test_folder = 'C:/Users/HP/Desktop/face_emotion_github/Data/Fer2013/archive/test/'
train_images, train_features, train_labels = load_images()
test_images, test_features, test_labels = load_images(test_folder)

# Define the neural network architecture
input_layer = Input(shape=(48, 48, 3))
x = Conv2D(32, kernel_size=(3,3), activation='relu')(input_layer)
x = MaxPooling2D()(x)
x = Conv2D(64, kernel_size=(3,3), activation='relu')(x)
x = MaxPooling2D()(x)
x = Conv2D(128, kernel_size=(3,3), activation='relu')(x)
x = MaxPooling2D()(x)
x = Flatten()(x)
lbp_input = Input(shape=(26,))
hog_input = Input(shape=(324,))
gabor_input = Input(shape=(40,))
# concatenated = Concatenate()([x, lbp_input, hog_input, gabor_input])
# dense1 = Dense(256, activation='relu')(concatenated)
# dropout1 = Dropout(0.5)(dense1)
# dense2 = Dense(128, activation='relu')(dropout1)
# dropout2 = Dropout(0.5)(dense2)
# output_layer = Dense(num_classes, activation='softmax')(dropout2)
# model = Model(inputs=[input_layer, lbp_input, hog_input, gabor_input], outputs=output_layer)

# model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# model.summary()''

In [ ]:
# Or try this

In [ ]:
# Load training data and extract features
train_folder = 'C:/Users/HP/Desktop/face_emotion_github/Data/Fer2013/archive/train/'
train_images, train_labels, train_features = load_images(train_folder)

# Load testing data and extract features
test_folder = 'C:/Users/HP/Desktop/face_emotion_github/Data/Fer2013/archive/test/'
test_images, test_labels, test_features = load_images(test_folder)

# Define the neural network architecture
input_layer = Input(shape=(48, 48, 3))
x = Conv2D(32, kernel_size=(3,3), activation='relu')(input_layer)
x = MaxPooling2D()(x)
x = Conv2D(64, kernel_size=(3,3), activation='relu')(x)
x = MaxPooling2D()(x)
x = Conv2D(128, kernel_size=(3,3), activation='relu')(x)
x = MaxPooling2D()(x)
x = Flatten()(x)

lbp_input_layer = Input(shape=(26,))
hog_input_layer = Input(shape=(3780,))
gabor_input_layer = Input(shape=(32,))

concat_layer = concatenate([x, lbp_input_layer, hog_input_layer, gabor_input_layer])
hidden_layer = Dense(512, activation='relu')(concat_layer)
output_layer = Dense(7, activation='softmax')(hidden_layer)

model = Model(inputs=[input_layer, lbp_input_layer, hog_input_layer, gabor_input_layer], outputs=output_layer)

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model.summary()
